# ML for everyone


นำเบ้า library ที่ใช้งาน

In [65]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler 

In [41]:
cols = ["fLength","fWidth","fSize","fConc","fConl","fAsym","FM3Long","FM3Trans","fAlpha","fDist","class"]
df = pd.read_csv('magic04.data',names=cols)
df.head()

,fLength,fWidth,fSize,fConc,fConl,fAsym,FM3Long,FM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [42]:
df["class"] = (df["class"]=="g").astype(int)
df

,fLength,fWidth,fSize,fConc,fConl,fAsym,FM3Long,FM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1
...,...,...,...,...,...,...,...,...,...,...,...
19015,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,0
19016,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,0
19017,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,0
19018,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,0


In [ ]:
# colume fLength where class ==1
# df[df['class']==1]['fLength']
# plt.hist(df[df['class']==1]['fLength'],color="blue",label="gamma",alpha=0.7,density=True)

for label in cols[:-1] :
    plt.hist(df[df['class']==1][label],color="blue",label="gamma",alpha=0.7,density=True)
    plt.hist(df[df['class']==0][label],color="red",label="hadron",alpha=0.7,density=True)
    plt.title(label)
    plt.ylabel('Probability')
    plt.xlabel(label)
    plt.legend()
    plt.show()


In [77]:
# [11412, 15216]
train,validate,test = np.split(df.sample(frac=1), [int(0.6*len(df)),int(0.8*len(df)) ])

c:\Users\sorawit_frappet\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [78]:
def scale_dataset(dataframe,oversample=False) :
    X = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values
    scatter = StandardScaler()
    X = scatter.fit_transform(X)
    if oversample :
        ros = RandomOverSampler()
        X,y = ros.fit_resample(X,y)
    data= np.hstack((X,np.reshape(y,(-1,1))))
    return data,X,y

In [79]:
train, X_train, y_train = scale_dataset(train, oversample=True)
validate, X_valid, y_valid = scale_dataset(validate, oversample=True)
test, X_test, y_test = scale_dataset(test, oversample=True)

## KNN

In [80]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train,y_train)
y_pred = knn_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.70      0.77      2482
           1       0.75      0.89      0.81      2482

    accuracy                           0.80      4964
   macro avg       0.81      0.80      0.79      4964
weighted avg       0.81      0.80      0.79      4964



## Naive Bayes

In [82]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model.fit(X_train,y_train)
y_pred = nb_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.42      0.55      2482
           1       0.61      0.89      0.72      2482

    accuracy                           0.66      4964
   macro avg       0.70      0.66      0.64      4964
weighted avg       0.70      0.66      0.64      4964



## Log Regression

In [83]:
from sklearn.linear_model import LogisticRegression
lg_model = LogisticRegression()
lg_model.fit(X_train,y_train)
y_pred = lg_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.73      0.77      2482
           1       0.75      0.83      0.79      2482

    accuracy                           0.78      4964
   macro avg       0.78      0.78      0.78      4964
weighted avg       0.78      0.78      0.78      4964



## SVM

In [85]:
from sklearn.svm import SVC 
sv_model = SVC()
sv_model.fit(X_train,y_train)
y_pred = sv_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.80      0.84      2482
           1       0.82      0.90      0.86      2482

    accuracy                           0.85      4964
   macro avg       0.85      0.85      0.85      4964
weighted avg       0.85      0.85      0.85      4964

